In [23]:
#include <type_traits>
#include <mutex>
#include <limits>
#include <initializer_list>
#include <vector>

## 6.1 constexpr variable

In [2]:
constexpr double PI = 3.1415926535;
constexpr auto area = PI * 2.0 * 2.0;

In [3]:
template<typename T> 
constexpr bool is_integral_v = std::is_integral_v<T>;

In [4]:
static_assert( is_integral_v<int> );

In [5]:
static_assert( not is_integral_v<void>);

In [6]:
template<char c>
constexpr bool is_digit = (c >= '0' and c <= '9');

In [7]:
static_assert(is_digit<'0'>);

In [8]:
static_assert(not is_digit<'x'>);

## 6.2 constinit

In [9]:
// constinit std::mutex g_mtx;

constinit is not supported Cling with gcc 12.2.0 on Debian.

## 6.3 fold expression

In [10]:
template<size_t...Is>
constexpr size_t rsum = (Is + ... + 0);
template<size_t...Is>
constexpr size_t lsum = (0 + ... + Is);

In [11]:
lsum<100>

(const unsigned long) 100


In [12]:
lsum<100, 1>

(const unsigned long) 101


In [13]:
rsum<1, 100>

(const unsigned long) 101


In [14]:
template<size_t...Is>
constexpr int rsub = ( Is - ... - 0);
template<size_t...Is>
constexpr int lsub = (0 - ... - Is);

In [15]:
rsub<100>

(const int) 100


In [16]:
lsub<100>

(const int) -100


In [17]:
rsub<100, 10>

(const int) 90


In [18]:
lsub<100, 10>

(const int) -110


## 6.4 constexpr function

In [19]:
constexpr int min(std::initializer_list<int> xs){
    int low = std::numeric_limits<int>::max();
    for(auto x : xs){
        if(x < low){
            low = x;
        }
    }
    return low;
}

In [20]:
min({1,1,3,4});

(int) 1


It seems cling does not support `consteval`:
```C++
consteval int min_eval(std::initializer_list<int> xs){
    int low = std::numeric_limits<int>::max();
    for(auto x : xs){
        if(x < low){
            low = x;
        }
    }
    return low;
}
```

Compilation time allocated memory should be freed before the end of compilation.

```C++
#include <vector>
#include <array>
#include <iostream>

constexpr std::vector<int> sievePrime(int n){
    std::vector<bool> marked(n+1, true);
    for(int p = 2; p * p < n; ++p){
        if(marked[p]){
            for(int i = p * p; i <= n; i += p){
                marked[i] = false;
            }
        }
    }
    
    std::vector<int> result;
    for(int p = 2; p < n; ++p){
        if(marked[p]){
            result.push_back(p);
        }
    }
    return result;
}

constexpr std::size_t primeCount(int n){
    return sievePrime(n).size();
}

template<int n>
consteval auto savePrimeToArray(){
    std::array<int, primeCount(n)> result;
    auto primes = sievePrime(n);
    std::copy(primes.begin(), primes.end(), result.begin());
    return result;
}

int main(void){
    static_assert(primeCount(100) == 25);
    auto primes = savePrimeToArray<100>();
    for(auto e: primes){
        std::cout << e << std::endl;
    }
}
```